In [ ]:
!pip install factory_boy

     |████████████████████████████████| 1.2MB 9.4MB/s 


# Factory Boy

## [Factory Boy Basics](https://factoryboy.readthedocs.io/en/latest/introduction.html)
The purpose of `factory_boy` is to provide a default way of getting a new instance, while still being able to override some fields on a per-call basis.



In [ ]:
%%writefile objects.py
class Account:
    def __init__(self, username, email, date_joined):
        self.username = username
        self.email = email
        self.date_joined = date_joined

    def __str__(self):
        return '%s (%s) joined at %s' % (self.username, self.email, self.date_joined)


class Profile:

    GENDER_MALE = 'm'
    GENDER_FEMALE = 'f'
    GENDER_UNKNOWN = 'u'  # If the user refused to give it

    def __init__(self, account, gender, firstname, lastname, planet='Earth'):
        self.account = account
        self.gender = gender
        self.firstname = firstname
        self.lastname = lastname
        self.planet = planet

    def __str__(self):
        return '%s %s (%s) joined at (s)' % (
            self.firstname,
            self.lastname,
            self.account.username,
        )

Writing objects.py


Factories declare a set of attributes used to instantiate an object, whose class is defined in the `class Meta`’s `model` attribute:
- Subclass `factory.Factory` (or a more suitable subclass)
- Add a `class Meta:` block
- Set its `model` attribute to the target class
- Add defaults for keyword args to pass to the associated class' `__init__` method

In [ ]:
# factory code
# Factories declare a set of attributes used to instantiate an object

import datetime
import factory
import random

import objects

class AccountFactory(factory.Factory):
    class Meta:  # use the Meta class to tell your factory which model to use.
        model = objects.Account  # point to target class

    ### declare attributes used to instantiate objects.Account ###
    # sequence: # john0 -> john1 -> john2 ...
    username = factory.Sequence(lambda n: 'john%s' % n)  
    # LazyAttribute: john0@example.org -> john1@example.org -> ...
    #   receive a function taking the object being built and returning the value for the field:
    email = factory.LazyAttribute(lambda o: '%s@example.org' % o.username)
    # LazyFunction: receive a function taking no argument and returning the value for the field
    date_joined = factory.LazyFunction(datetime.datetime.now)
# >>> user_accout = AccountFactory()

class ProfileFactory(factory.Factory):
    class Meta:
        model = objects.Profile

    account = factory.SubFactory(AccountFactory)
    gender = factory.Iterator([objects.Profile.GENDER_MALE, objects.Profile.GENDER_FEMALE])
    firstname = 'John'
    lastname = 'Doe'

In [ ]:
%load_ext autoreload
%autoreload 2
import objects

In [ ]:
# instantiated obj from class
acc = objects.Account("chet", "chet@gmail.com", "2021-04-14")
print(acc)

chet (chet@gmail.com) joined at 2021-04-14


In [ ]:
# instantiate obj from Factory Boy
acc_factory = AccountFactory()  # 如果是ORM的话, 这个就相当于 Account.objects.create(username=xx, email=xx, date_joined=xx)
print(acc_factory)

john1 (john1@example.org) joined at 2021-04-14 21:54:49.106545


## [Use factory boy with ORMs](https://factoryboy.readthedocs.io/en/stable/orms.html#using-factory-boy-with-orms)


```python
# gohan/tests/factories.py
import factory

from gohan.models import Hash2Sku, Sku2Hash


class Sku2HashFactory(factory.django.DjangoModelFactory):
    class Meta:
        model = Sku2Hash

    sku_id = factory.Faker("name")
    hash_values = factory.Faker("pylist", value_types=str, nb_elements=16)


class Hash2SkuFactory(factory.django.DjangoModelFactory):
    class Meta:
        model = Hash2Sku

    hash_value = factory.Faker("pystr", max_chars=16)
    sku_ids = factory.Faker("pylist", value_types="name")
```

**Below is an example of using aboved defined Factory in Pytest:**

`@pytest.mark.django_db`:
- This is used to mark a test function as requiring the database. It will ensure the __database is set up correctly for the test__. 
  - Each test will run __in its own transaction__ which will __be rolled back__ at the end of the test. This behavior is the same as Django’s standard `TestCase` class.

```python
# gohan/tests/gohan/test_views.py
class TestDuplicate:
    @pytest.mark.django_db
    def test_get_duplicates_success(self, client):
        Sku2HashFactory(sku_id="batman", hash_values=["123", "456"])
        Sku2HashFactory(sku_id="robin", hash_values=["123"])
        Hash2SkuFactory(hash_value="123", sku_ids=["batman"])
        Hash2SkuFactory(hash_value="456", sku_ids=["batman", "robin"])
        resp = client.post(
            "/v1/duplicate/",
            content_type="application/json",
            data={
                "objects": [
                    {
                        "sku_id": "batman",
                        "images": [{"url": "https://www.example.com/mock_image.jpeg"}],
                    },
                    {
                        "sku_id": "robin",
                        "images": [{"url": "https://www.example.com/mock_image.jpeg"}],
                    },
                    {
                        "sku_id": "superman",
                        "images": [{"url": "https://www.example.com/mock_image.jpeg"}],
                    },
                ]
            },
        )
        assert resp.status_code == 200

        assert resp.data["results"][0]["reference_sku_id"] == "batman"
        assert resp.data["results"][0]["duplicate_sku_ids"][0]["sku_id"] == "robin"

        assert resp.data["results"][1]["reference_sku_id"] == "robin"
        assert resp.data["results"][1]["duplicate_sku_ids"][0]["sku_id"] == "batman"

        assert resp.data["results"][2]["reference_sku_id"] == "superman"
        assert resp.data["results"][2]["duplicate_sku_ids"][0]["sku_id"] is None
```
`lyst-gohan/tests/gohan/test_models.py` also have good examples.

# Faker
https://zetcode.com/python/faker/

In [1]:
!pip install factory_boy

     |████████████████████████████████| 1.2 MB 8.5 MB/s 


In [2]:
from faker import Faker
fake = Faker()

In [3]:
fake.name()

'Amanda Avila'

In [4]:
fake.uuid4()

'51e7d233-c160-41fc-8f81-0bd187a4c31f'

In [5]:
fake.random.randint(1, 2)

2

In [21]:
fake.random_element(elements=(["gender.men"], ["gender.women"], ["gender.men", "gender.women"]))

['gender.men']

In [6]:
# https://faker.readthedocs.io/en/master/providers/faker.providers.python.html#faker.providers.python.Provider.pylist
fake.pylist(value_types=str, nb_elements=16)

['bIxIutvLdEWzQnksnTMU',
 'EGQiwgWTIvIKXWFEeEkj',
 'oZQsLSAKgWsAQmjXIUHp',
 'WBJnoDMOHVubnPdqAEnG',
 'fHTByzxbGxEzscNjXDkG',
 'cORLjkinOQSIMBUNqkPd',
 'AcXpYZsqgMvDMMCWJDgR',
 'DXJiYzgjQjHuqEqpNuzj',
 'zPxXQiYABHvfetRNoUYb',
 'opglbErDlWaXLdMgsZkB',
 'AeoyndmKpASnRfxJVYgU',
 'AWaatpNdKDwkNamjSina',
 'fhuzpYzyhILgwKGyuYNO',
 'ZJdcSeQkIFLrsucCIRaE',
 'HSadPtoAQrYFdkdJkjSs',
 'rBBCWGgpZBjmsxjQweoI']

In [7]:
fake.pylist(value_types="name", nb_elements=16)

['Sabrina Henderson',
 'Janet Black',
 'Margaret Curtis',
 'Christopher Levine',
 'David Yu',
 'Adam Chandler',
 'Laura Johnson',
 'Curtis Peterson',
 'Jennifer Estrada MD',
 'Frank Wilkinson',
 'Henry Maynard',
 'Paige Jackson',
 'Penny Gonzalez',
 'Casey Whitehead',
 'Joseph Brennan',
 'Carrie Fisher',
 'Adam Lee']

In [ ]:
fake.pystr(max_chars=16)

'UfvodZAQCaMMkGUo'

In [ ]:
import uuid
v = uuid.uuid4()
v

UUID('b36f5e3a-79a1-4357-af5e-a6a7578f8a02')

In [ ]:
str(v)

'b36f5e3a-79a1-4357-af5e-a6a7578f8a02'

In [ ]:
fake.pystr_format("{{random_int}}")

'5565'

## faker in factory_boy
source code: https://factoryboy.readthedocs.io/en/stable/_modules/factory/faker.html
  ``` python  
    """Wrapper for 'faker' values.

    Args:
        provider (str): the name of the Faker field
        locale (str): the locale to use for the faker

        All other kwargs will be passed to the underlying provider
        (e.g ``factory.Faker('ean', length=10)``
        calls ``faker.Faker.ean(length=10)``)

    Usage:
        >>> foo = factory.Faker('name')
    """
  ```
  Examples: `lyst-gohan/tests/factories.py`

In [ ]:
import factory